In [1]:
##### Part 0: housekeeping 
import csv
import pandas as pd
import copy
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import itertools
%pprint

Pretty printing has been turned OFF


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [3]:
def rearrange_col(dataframe, col_to_move, reference_col, right=True):
    col_list = dataframe.columns.values.tolist()
    col_list2 = [x for x in col_list if x != col_to_move]
    reference_idx = col_list2.index(reference_col)
    if right==True:
        col_list3 = []
        for y in col_list2:
            col_list3.append(y)
            if y == reference_col:
                col_list3.append(col_to_move)
        return dataframe[col_list3]
    else:
        col_list3 = []
        for y in col_list2:
            col_list3.append(y)
            if y == reference_col:
                col_list3.insert(-2, col_to_move)
        return  dataframe[col_list3]

def PopSamples(pool, num_subset):
    np.random.shuffle(pool)
    return pool[:num_subset]

def ttest_is_significant (series1, series2, threshold=0.05, equal_var=False):
    #statistic = stats.ttest_ind(series1, series2, equal_var=equal_var).statistic
    #pvalue = stats.ttest_ind(series1, series2, equal_var=equal_var).pvalue
    ret = stats.ttest_ind(series1, series2, equal_var=equal_var)
    if ret.pvalue > threshold: 
        result = 1  # pass, i.e, 'Insignificant'
    else:
        result = 0  # fail, i.e, 'Significantly different distribution'
    return result

In [4]:
##### 
I_Mean_RT_m = 784.10
I_Mean_RT_sd = 134.98
I_Mean_Acc_m = 0.84
I_Mean_Acc_sd = 0.20
AgeOfAcqsn_m = 9.71
AgeOfAcsqn_sd = 2.96
LogFreqHAL_m = 6.16
LogFreqHAL_sd = 2.40

In [47]:
starting_list = '/Users/songheekim/Google Drive/Primary/Projects/VerbVector/final_list/50_55_55_candidate1_wb.csv'
df_raw = pd.read_csv(starting_list, index_col='Lemma')
df_init = pd.read_csv(starting_list, index_col='Lemma')
print('df_init:', df_init.shape[0], '  **before cleaning**')

### z-scoring
df_init['I_Mean_RT_z'] = (df_init['I_Mean_RT']-I_Mean_RT_m)/I_Mean_RT_sd
df_init['I_Mean_Accuracy_z'] = (df_init['I_Mean_Accuracy']-I_Mean_Acc_m)/I_Mean_Acc_sd
df_init['AgeofAcqsn_z'] = (df_init['AgeofAcqsn']-AgeOfAcqsn_m)/AgeOfAcsqn_sd
df_init['LogFreqHAL_z'] = (df_init['LogFreqHAL']-LogFreqHAL_m)/LogFreqHAL_sd

#df_init = df_init.loc[df_init['badverb']=='n'] ## subset so that df_init only includes 'good' verbs
df_in = df_init.loc[df_init['in']==1] ### in==0: potentially ambiguous verbs
df_synmatch = df_in.loc[df_in['syn_matched']==1]
df_semmatch = df_in.loc[df_in['sem_matched']==1]
sem_tab = pd.crosstab(df_in.SemClass, df_in.SynClass, margins=True).sort_values(by='All', ascending=False)

print ('df_init: ', df_init.shape[0], ' **after cleaning**')
print ('df_in: ', df_in.shape[0])
print ('df_synmatch: ', df_synmatch.shape[0])
print ('df_semmatch: ', df_semmatch.shape[0])

#z=1.282 ##90th percentile

df_init: 403   **before cleaning**
df_init:  403  **after cleaning**
df_in:  384
df_synmatch:  188
df_semmatch:  84


In [ ]:
df_init: 403   **before cleaning**
df_init:  388  **after cleaning**
df_in:  384
df_synmatch:  188
df_semmatch:  84

In [ ]:
#### let's mark some marginal ones. 
### Part1: based on absolute criteria (ELP statistics)
# marginal = {}
# criteria = ['I_Mean_RT_z','I_Mean_Accuracy_z', 'AgeofAcqsn_z', 'LogFreqHAL_z']
# for c in criteria: 
#     marginal[c] = dict.fromkeys(['high10', 'low10'])
#     #print ('==========', c, '=========')
#     #print ('lowest 10%', df_init.loc[df_init[c] <= -1.282].index.tolist())
#     marginal[c]['low10'] = df_init.loc[df_init[c] <= -1.282].index.tolist()
#     #print ('highest 10%', df_init.loc[df_init[c] >= 1.282].index.tolist())
#     marginal[c]['high10'] = df_init.loc[df_init[c] >= 1.282].index.tolist()

# 'disintegrate' in marginal['LogFreqHAL_z']['low10']

#import scipy.stats as st
#st.norm.ppf(0.999) ## from percentile to z score
# st.norm.cdf(1) ## from z score to percentile


In [15]:
def get_absolute_percentile(zscore, polarity): 
    '''function that take in Z score, return: 
       the proportion of data larger than the z-score (if polarity='upper')
       the proportion of data smaller than the z-score (if polarity='lower')'''
    import scipy.stats as st
    if polarity == 'upper':
        absolute_percentile = 1 - st.norm.cdf(zscore)
    elif polarity == 'lower':
        absolute_percentile = st.norm.cdf(zscore)
    else:
        raise Exception('Polarity should be either \'upper\' or \'lower\'')
    return round(absolute_percentile,4)

In [48]:
#### let's find some marginal ones. 
############### PART1: based on absolute criteria (ELP statistics)##########
df_init_dict = df_init.to_dict(orient='index')
# print ('before==============')
# print (df_init_dict['writhe'], '\n')

###### create a metric, i.e.,'absolute marginality' 
import scipy.stats as st
for k, v in df_init_dict.items():
    if v['I_Mean_Accuracy_z'] <= st.norm.ppf(0.2): 
        #print (k, 'will be dropped')
        v['abs_marginality'] = 0   ### Mark it marginal if LD MeanAccuracy is below 20% cutoff
    else:
        rt_marginal = get_absolute_percentile(v['I_Mean_RT_z'], 'upper') #how extremely HIGH the RT is
        acc_marginal = get_absolute_percentile(v['I_Mean_Accuracy_z'], 'lower') #how extremely LOW the accuracy is
        acqsn_marginal = get_absolute_percentile(v['AgeofAcqsn_z'], 'upper') 
        freq_marginal = get_absolute_percentile(v['LogFreqHAL_z'], 'lower')
        v['abs_marginality'] = round((rt_marginal + acc_marginal + acqsn_marginal + freq_marginal)/4, 4)

# print ('absolute added==============')
# print (df_init_dict['writhe'], '\n')

################ PART2: based on relative ranking of marginality

#df_init['I_Mean_RT'].sort_values(ascending=False)[:38] ###member with largest 10% RT 
#df_init['I_Mean_Accuracy'].sort_values(ascending=True)[:38] ### members with lowest 10% Accuracy 
#df_init['AgeofAcqsn'].sort_values(ascending=False)[:38] ### member with highest 10% Age of Acquisition
#df_init['LogFreqHAL'].sort_values(ascending=True)[:38] ### members with lowest 10% LogFreqHAL
rt_rel_marginal = df_init['I_Mean_RT'].sort_values(ascending=False)[:38].index.tolist()
accuracy_rel_marginal = df_init['I_Mean_Accuracy'].sort_values(ascending=True)[:38].index.tolist()
ageofacqsn_rel_marginal = df_init['AgeofAcqsn'].sort_values(ascending=False)[:38].index.tolist()
logfreqhal_rel_marginal = df_init['LogFreqHAL'].sort_values(ascending=True)[:38].index.tolist()

rt_accuracy = [i for i in accuracy_rel_marginal if i in rt_rel_marginal] 
rt_accuracy_freq = [i for i in logfreqhal_rel_marginal if i in rt_accuracy]
rt_accuracy_freq_age = [i for i in logfreqhal_rel_marginal if i in rt_accuracy_freq]
marginal_relative = sorted(rt_accuracy_freq_age)

for k, v in df_init_dict.items():
    if k in marginal_relative: 
        v['rel_marginality'] = 'y'   ### Mark it marginal if found in marginal_relative list
    else:
        v['rel_marginality'] = 'n'
        
# print ('relative added==============')
# print (df_init_dict['writhe'], '\n')

### convert the dict to a df and save
df_init_updated = pd.DataFrame.from_dict(df_init_dict, orient='index')

### find what's common between relative and absolute 
marginal_absolute = df_init_updated.sort_values(by='abs_marginality')[:20].index.tolist()
#print ('marginal_absolute', marginal_absolute)
marginal_absolute_relative = [i for i in marginal_relative if i in marginal_absolute]
#print ('marginal by any criterion:', marginal_absolute_relative)


#### exclude verbs that are marginal by either criterion
for verb in marginal_absolute_relative: 
    print (verb, df_init_updated.loc[verb, 'in'])
    df_init_updated.loc[verb, 'in'] = 0
    print (verb, df_init_updated.loc[verb, 'in'])

ensue 1
ensue 0
recur 1
recur 0
seep 1
seep 0
swerve 1
swerve 0
writhe 1
writhe 0


In [55]:
#df_out = df_init_updated.sort_values(by=['badverb', 'in', 'syn_matched', 'SynClass', 'GrandIndex'], ascending=[True, False, False, False, True])
df_out.to_csv('/Users/songheekim/Google Drive/Primary/Projects/VerbVector/Verblists/List300_v0.csv')

In [16]:
#get_absolute_percentile(-1.645, 'lower')

In [ ]:
####### draw venn diagram
# 
# musicians = {from venn import venn
# rel_marginal = {
#     'RT': set(rt_rel_marginal),
#     'Accuaracy' : set(accuracy_rel_marginal), 
#     'AgeOfAcqsn': set(ageofacqsn_rel_marginal), 
#     'LogFreqHAL': set(logfreqhal_rel_marginal)
# }
#     "Members of The Beatles": {"Paul McCartney", "John Lennon", "George Harrison", "Ringo Starr"},
#     "Guitarists": {"John Lennon", "George Harrison", "Jimi Hendrix", "Eric Clapton", "Carlos Santana"},
#     "Played at Woodstock": {"Jimi Hendrix", "Carlos Santana", "Keith Moon"}
# }
venn(rel_marginal)

In [ ]:
# ages = [5, 12, 17, 18, 24, 32]

# def myFunc(x):
#     if x < 18:
#         return False
#     else:
#         return True

# adults = filter(myFunc, ages)

# for a in adults:
#     print(a)

# aa=5
#globals()['aa'] + 10
#locals().items()

In [ ]:
df_dict_updated = pd.DataFrame.from_dict(df_init_dict, orient='index')
df_dict_updated_sort = df_dict_updated.sort_values(by='marginality', ascending=True)
marginal_absolute = sorted(df_dict_updated_sort[:10].index.tolist())
# for k, v in df_init_dict.items():
#     print (k, v['marginality'])

In [ ]:
############ CHECK IF SYNTACTIC CATEGORIES ARE MATCHED FOR LOW LEVEL QUALITIES 
syncat = ['unergative', 'unaccusative', 'transitive']
print ('df_synmatch:', df_synmatch.shape)

unergative = df_synmatch.loc[df_synmatch['SynClass']=='unergative']
unaccusative = df_synmatch.loc[df_synmatch['SynClass']=='unaccusative']
transitive = df_synmatch.loc[df_synmatch['SynClass']=='transitive']

print ('unergative:', unergative.shape[0])
print ('unaccusative:', unaccusative.shape[0])
print ('transitive:', transitive.shape[0])

cols_test = ['LogFreqHAL', 'AgeofAcqsn', 'Length', 'Ortho_N', 'Phono_N', 'OLD', 'PLD', 'NPhon', 'NSyll', \
            'I_Mean_RT', 'I_Mean_Accuracy', 'I_NMG_Mean_RT', 'I_NMG_Mean_Accuracy', 'N2_F', 'N3_F']
pairs1 = [('unergative', 'unaccusative'), ('unaccusative', 'transitive'), ('transitive', 'unergative')]

for c in cols_test:
    print ('===========', c, '============')
    for p in pairs1:
        x = ttest_is_significant(df_synmatch.loc[df_synmatch['SynClass']==p[0]][c].tolist(), \
                         df_synmatch.loc[df_synmatch['SynClass']==p[1]][c].tolist(), \
                         threshold=0.05, equal_var=False)
        if x == 0:
            print (c, p, 'p < 0.05')
            print ('-->', p[0], round(df_synmatch.loc[df_synmatch['SynClass']==p[0]][c].mean(),2))
            print ('-->', p[1], round(df_synmatch.loc[df_synmatch['SynClass']==p[1]][c].mean(),2))
print ('syntactic test finished')

In [ ]:
'live' in unergative.index

In [ ]:
### metric: AgeofAcqsn, LogFreqHAL, I_Mean_RT, I_Mean_Accuracy